<a href="https://colab.research.google.com/github/Plashka320/-/blob/main/StatTests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оглавление

1. [Вступление и истоки проблемы](#section1)
2. [Самодельный инструмент](#section2)
3. [Использование SciPy tests](#section3)
4. [О смысле p-value, корректности и мощности](#section4)
5. [Перестановочные тесты](#section5)
6. [Bootstrap](#section6)
7. [Бакетное сэмплирование](#section7)
8. [Заключение](#section8)


# <a name="section1"></a> 1. Вступление и истоки проблемы

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import scipy.stats

%matplotlib inline

Сгенерируем данные с одинаковыми средними

In [ ]:
data_fst = np.random.normal(10, 10, size=100)
data_snd = np.random.normal(10, 1, size=100)

Если мы посмотрим на средние значения выборок, то они конечно же будут различны

In [ ]:
print(data_fst.mean())
print(data_snd.mean())

Но если средние разные, то числа тоже могут не сильно отличаться

In [ ]:
data_fst = np.random.normal(10.5, 10, size=100)
data_snd = np.random.normal(10, 1, size=100)

print(data_fst.mean())
print(data_snd.mean())

Таким образом, перед нами встаёт задача научиться отличать случайные отличия и  неслучайные

# <a name="section2"></a> 2. Самодельный инструмент

Можно воспользоваться ЦПТ и получить доверительный интервал для среднего

**Напоминание ЦПТ:**
Пусть $\xi_1, \xi_2, \dots$ - независимые одинаково распределённые случайные величины с конечной дисперсией $D \xi_1$ и мат. ожиданием $E \xi_1$, тогда

$\sqrt{\frac{n}{D \xi_1}} \left(\frac{\sum\limits_{i=1}^{n} \xi_i}{n} - E \xi_1\right) \to_{n \to \infty} \mathcal{N}(0, 1)$

Отсюда можно сделать вывод, что при больших $n$ величина $\frac{\sum\limits_{i=1}^{n} \xi_i}{n}$ будет распределена примерно как $\mathcal{N}(E \xi_1, \frac{D \xi_1}{n})$

Если бы мы знали $D \xi_1$, то отсюда можно получить интервал на $E \xi_1$:
$$
P\left(E \xi_1 \in \left[\frac{\sum\limits_{i=1}^{n} \xi_i}{n} - 1.96 \sqrt{\frac{D \xi_1}{n}}, \frac{\sum\limits_{i=1}^{n} \xi_i}{n} + 1.96 \sqrt{\frac{D \xi_1}{n}}\right]\right) \approx 0.95
$$

$D \xi_1$ можно оценить по выборке

In [ ]:
def get_95_interval(data, sigma=None):
    mean = data.mean()
    if sigma is None:
        sigma = data.std()
    err = sigma * 1.96 / np.sqrt(len(data))
    return mean - err, mean + err

Проверим, действительно ли наш доверительный интервал содержит истинное среднее значение в 95% случаев:
Сделаем 10 тыс. симуляций, в каждой возьмем случайную выборку, посчитаем выборочное среднее и его доверительный интервал, после чего посмотрим, попадает ли истинное среднее в него. В итоге посмотрим, в каком % симуляций истинное среднее попало в ДИ

In [ ]:
hits = 0.
count = 0.
true_mean = 10.

for _ in range(10000):
    data = np.random.normal(true_mean, 10, size=100)
    lower_bound, upper_bound = get_95_interval(data)
    count += 1
    if lower_bound <= true_mean <= upper_bound:
        hits += 1

print(hits / count)

Однако, если данных мало, то оценка неправильна

In [ ]:
hits = 0.
count = 0.
true_mean = 10.

for _ in range(10000):
    data = np.random.normal(true_mean, 10, size=10)
    lower_bound, upper_bound = get_95_interval(data)
    count += 1
    if lower_bound <= true_mean <= upper_bound:
        hits += 1

print(hits / count)

Подставим реальное значение дисперсии и всё снова заработает

In [ ]:
hits = 0.
count = 0.
true_mean = 10.

for _ in range(10000):
    data = np.random.normal(true_mean, 10, size=10)
    lower_bound, upper_bound = get_95_interval(data, sigma=10)
    count += 1
    if lower_bound <= true_mean <= upper_bound:
        hits += 1

print(hits / count)

Итак, у нас две выборки с гигантской разницей в среднем

In [ ]:
data_fst = np.random.normal(10.5, 10, size=100)
data_snd = np.random.normal(100, 1, size=100)

print(get_95_interval(data_fst))
print(get_95_interval(data_snd))

Конечно же интервалы не пересекаются

Однако, если разница небольшая, то интервалы могут пересечься

In [ ]:
data_fst = np.random.normal(11, 10, size=500)
data_snd = np.random.normal(10, 1, size=100)

print(get_95_interval(data_fst))
print(get_95_interval(data_snd))

Давайте посмотрим, как часто такой критерий (доверительные интервалы не пересекаются) будет находить различие в случае, если оно есть. В некотором смысле это мощность критерия

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(11, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    lower_fst, upper_fst = get_95_interval(data_fst)
    lower_snd, upper_snd = get_95_interval(data_snd)

    count += 1
    if not (lower_fst <= lower_snd <= upper_fst or lower_fst <= upper_snd <= upper_fst):
        hits += 1

print(hits / count)

Также проверим, как часто различие будет обнаруживаться, если его нет

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(10, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    lower_fst, upper_fst = get_95_interval(data_fst)
    lower_snd, upper_snd = get_95_interval(data_snd)

    count += 1
    if not (lower_fst <= lower_snd <= upper_fst or lower_fst <= upper_snd <= upper_fst):
        hits += 1

print(hits / count)

Также проверим, как часто различие будет обнаруживаться, если он есть, но очень маленький

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(10.1, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    lower_fst, upper_fst = get_95_interval(data_fst)
    lower_snd, upper_snd = get_95_interval(data_snd)

    count += 1
    if not (lower_fst <= lower_snd <= upper_fst or lower_fst <= upper_snd <= upper_fst):
        hits += 1

print(hits / count)

У нас очень маленькая ошибка первого рода (правда мы хотели 0.05), но мощность тоже небольшая

**Вывод**: этот способ очень прост, но можно использовать более мощные и подконтрольные критерии

В простом варианте давайте вычтем две незавимых случайных величины друг из друга

$\mathcal{N}(a_1, \sigma_1^2) - \mathcal{N}(a_2, \sigma_2^2) \sim \mathcal{N}(a_1 - a_2, \sigma_1^2 + \sigma_2^2)$

То есть в нашем случае $\frac{\sum\limits_{i=1}^{n^1} \xi_i^1}{n^1} - \frac{\sum\limits_{i=1}^{n^2} \xi_i^2}{n^2}$ будет распределена примерно как $\mathcal{N}(E \xi_1^1 - E \xi_1^2, \frac{D \xi_1^1}{n^1} + \frac{D \xi_1^2}{n^2})$

Проверим те же параметры, что и в предыдущем случае

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(11, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    stat = (
        (data_fst.mean() - data_snd.mean())
        /
        np.sqrt(1. * data_fst.std() ** 2 / len(data_fst) + 1. * data_snd.std() ** 2 / len(data_snd))
    )

    count += 1
    if abs(stat) > 1.96:
        hits += 1

print(hits / count)

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(10, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    stat = (
        (data_fst.mean() - data_snd.mean())
        /
        np.sqrt(1. * data_fst.std() ** 2 / len(data_fst) + 1. * data_snd.std() ** 2 / len(data_snd))
    )
    count += 1
    if abs(stat) > 1.96:
        hits += 1

print(hits / count)

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(10.1, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    stat = (
        (data_fst.mean() - data_snd.mean())
        /
        np.sqrt(1. * data_fst.std() ** 2 / len(data_fst) + 1. * data_snd.std() ** 2 / len(data_snd))
    )
    count += 1
    if abs(stat) > 1.96:
        hits += 1

print(hits / count)

Это уже более похоже на то, чего нам бы хотелось

# <a name="section3"></a> 3. Использование SciPy tests

Каждый раз писать такие критерии было чересчур, тем более в модуле scipy всё уже есть

Вот например в scipy есть куча различных тестов, в которых уже все формулы написаны и надо просто вызвать функцию

In [ ]:
scipy.stats.ttest_ind(data_fst, data_snd).pvalue

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(11, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd).pvalue < 0.05:
        hits += 1

print(hits / count)

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(10, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd).pvalue < 0.05:
        hits += 1

print(hits / count)

сломалось(

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(11, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.05:
        hits += 1

print(hits / count)

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(10, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.05:
        hits += 1

print(hits / count)

Примерно как и ЦПТ, только не надо париться с формулами

Давайте другой тест возьмём, например, Мана-Уитни

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(11, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    count += 1
    if scipy.stats.mannwhitneyu(data_fst, data_snd, alternative='two-sided').pvalue < 0.05:
        hits += 1

print(hits / count)

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(10, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    count += 1
    if scipy.stats.mannwhitneyu(data_fst, data_snd, alternative='two-sided').pvalue < 0.05:
        hits += 1

print(hits / count)

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(0, 10, size=1000)
    data_snd = np.random.normal(0, 1, size=100)

    count += 1
    if scipy.stats.mannwhitneyu(data_fst, data_snd, alternative='two-sided').pvalue < 0.05:
        hits += 1

print(hits / count)

Думаете всё сломалось? Нет, просто у этого критерия другая нулевая гипотеза!

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(0, 1, size=1000)
    data_snd = np.random.normal(0, 1, size=100)

    count += 1
    if scipy.stats.mannwhitneyu(data_fst, data_snd, alternative='two-sided').pvalue < 0.05:
        hits += 1

print(hits / count)

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(0, 1, size=1000)
    data_snd = np.random.normal(1, 1, size=100)

    count += 1
    if scipy.stats.mannwhitneyu(data_fst, data_snd, alternative='two-sided').pvalue < 0.05:
        hits += 1

print(hits / count)

# <a name="section4"></a> 4. О смысле p-value, корректности и мощности

In [ ]:
pvalues = []
for _ in range(10000):
    data_fst = np.random.normal(10, 10, size=100)
    data_snd = np.random.normal(10, 1, size=100)
    pvalues.append(scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue)

plt.hist(pvalues)
plt.show()

Корректность критерия

In [ ]:
bad_pvalues = []
for _ in range(10000):
    data_fst = np.random.normal(10, 10, size=100)
    data_snd = np.random.normal(10, 1, size=100)
    bad_pvalues.append(scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.02)

print(np.mean(bad_pvalues))

Мощность критерия

In [ ]:
pvalues = []
for _ in range(10000):
    data_fst = np.random.normal(11, 10, size=100)
    data_snd = np.random.normal(10, 1, size=100)
    pvalues.append(scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue)

print((np.array(pvalues) < 0.05).mean())
plt.hist(pvalues)
plt.show()

Усилим различие

In [ ]:
pvalues = []
for _ in range(10000):
    data_fst = np.random.normal(12, 10, size=100)
    data_snd = np.random.normal(10, 1, size=100)
    pvalues.append(scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue)

print((np.array(pvalues) < 0.05).mean())
plt.hist(pvalues)
plt.show()

Увеличим выборку

In [ ]:
pvalues = []
for _ in range(10000):
    data_fst = np.random.normal(11, 10, size=1600)
    data_snd = np.random.normal(10, 1, size=1600)
    pvalues.append(scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue)

print((np.array(pvalues) < 0.05).mean())
plt.hist(pvalues)
plt.show()

In [ ]:
pvalues = []
for _ in range(10000):
    data_fst = np.random.normal(11, 2.5, size=100)
    data_snd = np.random.normal(10, 0.25, size=100)
    pvalues.append(scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue)

print((np.array(pvalues) < 0.05).mean())
plt.hist(pvalues)
plt.show()

# <a name="section5"></a> 5. Перестановочные тесты

In [ ]:
def my_test(data_fst, data_snd, iters=1000, plot=True):
    data = np.array(list(data_fst) + list(data_snd))
    size = len(data)
    values = []
    fst_ratio = len(data_fst) * 1. / size
    for _ in range(iters):
        mask = (np.random.random(size=size) < fst_ratio)
        values.append(data[mask].mean() - data[~mask].mean())

    if plot:
        plt.hist(values, bins=30)
        plt.show()

    return (np.abs(values) > np.abs(data_fst.mean() - data_snd.mean())).mean()

Корректно ли такое применение?

In [ ]:
data_fst = np.random.normal(0, 10, size=100)
data_snd = np.random.normal(0, 1, size=100)

print(scipy.stats.ttest_ind(data_fst, data_snd).pvalue)
print(my_test(data_fst, data_snd, iters=10))
print(my_test(data_fst, data_snd, iters=100))
print(my_test(data_fst, data_snd, iters=1000))
print(my_test(data_fst, data_snd, iters=10000))

In [ ]:
data_fst = np.random.normal(0, 10, size=100)
data_snd = np.random.normal(1, 1, size=100)

print(scipy.stats.ttest_ind(data_fst, data_snd).pvalue)
print(my_test(data_fst, data_snd, iters=10))
print(my_test(data_fst, data_snd, iters=100))
print(my_test(data_fst, data_snd, iters=1000))
print(my_test(data_fst, data_snd, iters=10000))

К сожалению нет :(

Давайте в этом убедимся :(

In [ ]:
hits = 0.
count = 0.

for _ in range(1000):
    data_fst = np.random.normal(11, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    count += 1
    if my_test(data_fst, data_snd, plot=False, iters=1000) < 0.05:
        hits += 1

print(hits / count)

In [ ]:
hits = 0.
count = 0.

for _ in range(1000):
    data_fst = np.random.normal(10, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    count += 1
    if my_test(data_fst, data_snd, plot=False, iters=1000) < 0.05:
        hits += 1

print(hits / count)

Как с Мана-Уитни, нулевая гипотеза это что распределения совпадают

In [ ]:
hits = 0.
count = 0.

for _ in range(1000):
    data_fst = np.random.normal(11, 1, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    count += 1
    if my_test(data_fst, data_snd, plot=False, iters=1000) < 0.05:
        hits += 1

print(hits / count)

In [ ]:
hits = 0.
count = 0.

for _ in range(1000):
    data_fst = np.random.normal(10, 1, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    count += 1
    if my_test(data_fst, data_snd, plot=False, iters=1000) < 0.05:
        hits += 1

print(hits / count)

Вот это уже неплохо

# <a name="section6"></a> 6. Bootstrap

In [ ]:
def my_bootstrap(data_fst, data_snd, iters=1000, plot=True):
    values = []
    for _ in range(iters):
        values.append(
            np.random.choice(data_fst, replace=True, size=len(data_fst)).m йгean()
            -
            np.random.choice(data_snd, replace=True, size=len(data_snd)).mean()
        )

    if plot:
        plt.hist(values, bins=30)
        plt.show()

    return np.percentile(values, [2.5, 97.5])

In [ ]:
data_fst = np.random.normal(0, 10, size=1000)
data_snd = np.random.normal(0, 1, size=100)

print(scipy.stats.ttest_ind(data_fst, data_snd).pvalue)
print(my_bootstrap(data_fst, data_snd, iters=1000, plot=True))

In [ ]:
data_fst = np.random.normal(1, 10, size=1000)
data_snd = np.random.normal(0, 1, size=100)

print(scipy.stats.ttest_ind(data_fst, data_snd).pvalue)
print(my_bootstrap(data_fst, data_snd, iters=1000, plot=True))

In [ ]:
hits = 0.
count = 0.

for _ in range(1000):
    data_fst = np.random.normal(11, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    lower, upper = my_bootstrap(data_fst, data_snd, plot=False, iters=1000)
    count += 1
    if not (lower < 0 < upper):
        hits += 1

print(hits / count)

In [ ]:
hits = 0.
count = 0.

for _ in range(1000):
    data_fst = np.random.normal(10, 10, size=1000)
    data_snd = np.random.normal(10, 1, size=100)

    lower, upper = my_bootstrap(data_fst, data_snd, plot=False, iters=1000)
    count += 1
    if not (lower < 0 < upper):
        hits += 1

print(hits / count)

# <a name="section7"></a> 7. Бакетное сэмплирование

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(10, 5, size=10000)
    data_snd = np.random.normal(10, 1, size=10000)

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.05:
        hits += 1

print(hits / count)

0.0511


In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(10.1, 5, size=10000)
    data_snd = np.random.normal(10, 1, size=10000)

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.05:
        hits += 1

print(hits / count)

0.5043


делаем бакеты

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(10, 5, size=10000).reshape((100, 100)).mean(axis=1)
    data_snd = np.random.normal(10, 1, size=10000).reshape((100, 100)).mean(axis=1)

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.05:
        hits += 1

print(hits / count)

0.053


In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(10.1, 5, size=10000).reshape((100, 100)).mean(axis=1)
    data_snd = np.random.normal(10, 1, size=10000).reshape((100, 100)).mean(axis=1)

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.05:
        hits += 1

print(hits / count)

0.4924


Можно и суммы

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(10, 5, size=10000).reshape((100, 100)).sum(axis=1)
    data_snd = np.random.normal(10, 1, size=10000).reshape((100, 100)).sum(axis=1)

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.05:
        hits += 1

print(hits / count)

0.0494


In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    data_fst = np.random.normal(10.1, 5, size=10000).reshape((100, 100)).sum(axis=1)
    data_snd = np.random.normal(10, 1, size=10000).reshape((100, 100)).sum(axis=1)

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.05:
        hits += 1

print(hits / count)

0.4857


А теперь усложним выборку

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    params_fst = np.random.uniform(-5, 5, size=1000)
    params_snd = np.random.uniform(-5, 5, size=1000)

    data_fst = np.random.normal(10, 5, size=10000).reshape((1000, 10)) + params_fst[:, np.newaxis]
    data_fst = data_fst.reshape((10000,))
    data_snd = np.random.normal(10, 1, size=10000).reshape((1000, 10)) + params_snd[:, np.newaxis]
    data_snd = data_snd.reshape((10000,))

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.05:
        hits += 1

print(hits / count)

0.3513


In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    params_fst = np.random.uniform(-1, 1, size=1000)
    params_snd = np.random.uniform(-1, 1, size=1000)

    data_fst = np.random.normal(10.1, 5, size=10000).reshape((1000, 10)) + params_fst[:, np.newaxis]
    data_fst = data_fst.reshape((10000,))
    data_snd = np.random.normal(10, 1, size=10000).reshape((1000, 10)) + params_snd[:, np.newaxis]
    data_snd = data_snd.reshape((10000,))

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.05:
        hits += 1

print(hits / count)

0.4901


беда(

In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    params_fst = np.random.uniform(-5, 5, size=1000)
    params_snd = np.random.uniform(-5, 5, size=1000)

    data_fst = np.random.normal(10, 5, size=10000).reshape((1000, 10)) + params_fst[:, np.newaxis]
    data_fst = data_fst.reshape((10000,)).reshape((100, 100)).mean(axis=1)
    data_snd = np.random.normal(10, 1, size=10000).reshape((1000, 10)) + params_snd[:, np.newaxis]
    data_snd = data_snd.reshape((10000,)).reshape((100, 100)).mean(axis=1)

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.05:
        hits += 1

print(hits / count)

0.0486


In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    params_fst = np.random.uniform(-5, 5, size=1000)
    params_snd = np.random.uniform(-5, 5, size=1000)

    data_fst = np.random.normal(10.1, 5, size=10000).reshape((1000, 10)) + params_fst[:, np.newaxis]
    data_fst = data_fst.reshape((10000,)).reshape((100, 100)).mean(axis=1)
    data_snd = np.random.normal(10, 1, size=10000).reshape((1000, 10)) + params_snd[:, np.newaxis]
    data_snd = data_snd.reshape((10000,)).reshape((100, 100)).mean(axis=1)

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.05:
        hits += 1

print(hits / count)

0.1091


In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    params_fst = np.random.uniform(-1, 1, size=1000)
    params_snd = np.random.uniform(-1, 1, size=1000)

    data_fst = np.random.normal(10.1, 5, size=10000).reshape((1000, 10)) + params_fst[:, np.newaxis]
    data_fst = data_fst.reshape((10000,)).reshape((100, 100)).mean(axis=1)
    data_snd = np.random.normal(10, 1, size=10000).reshape((1000, 10)) + params_snd[:, np.newaxis]
    data_snd = data_snd.reshape((10000,)).reshape((100, 100)).mean(axis=1)

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.05:
        hits += 1

print(hits / count)

0.4139


In [ ]:
hits = 0.
count = 0.

for _ in range(10000):
    params_fst = np.random.uniform(-100, 100, size=1000)
    params_snd = np.random.uniform(-100, 100, size=1000)

    data_fst = np.random.normal(10.1, 5, size=10000).reshape((1000, 10)) + params_fst[:, np.newaxis]
    data_fst = data_fst.reshape((10000,)).reshape((100, 100)).mean(axis=1)
    data_snd = np.random.normal(10, 1, size=10000).reshape((1000, 10)) + params_snd[:, np.newaxis]
    data_snd = data_snd.reshape((10000,)).reshape((100, 100)).mean(axis=1)

    count += 1
    if scipy.stats.ttest_ind(data_fst, data_snd, equal_var=False).pvalue < 0.05:
        hits += 1

print(hits / count)

0.0504


# <a name="section8"></a> 8. Заключение

- задача статистических критериев отличать случайность от неслучайности
- существуют различные критерии и их можно применить, используя модуль `scipy`
- существуют симуляционные тесты: перестановочные и бутстреп, в них можно реализовать произвольный источник случайности
- на больших данных полезно применять бакетные тесты – они здорово сокращают объём данных